# Booking.com Feature Extraction
---

Download the file BookingDotCom_HotelReviews.xlsx from Canvas. This file contains over 515,000 guest reviews and rating of almost 1500 hotels across Europe scraped from popular hotel reservation website Booking.com. The text data was cleaned by removing unicode and punctuation and transformed to lower case. No other preprocessing was done. More information on each field is provided in the "Data Description" tab of the Excel file.

        1. What are the top five hotel features (e.g., location, staff, etc.) that customers mention the most in positive reviews and top five features they mention most in negative reviews? Your identified features must make sense (e.g., "great" or "negative" are not features). (3 points)
        
        2. What are the top five features that customers prefer most if they are a solo traveler vs traveling with a group vs on a business trip vs a leisure trip vs traveling as a couple vs a family with young children. You will find these categories in the "Tags" column. There are a few more tags that we don't need. (2 points).

        3. What are the top five features customers like most and top five features they complain about most about hotels in United Kingdom, France, Italy, and Spain? Country information is available inside Hotel_Address. (2 points)
        
        4. Create a dashboard with the following plots; (1) "Top Five Hotels Overall" with consistently high ratings, (2) Bottom Five Hotels Overall" with consistently low ratings, (3) Five Most Improved Hotels" with the highest improvement in average ratings from 2015 to 2017, showing their average ratings for each of the three years. (0.5+0.5+2 points).

Write clear, compact, and understandable code with comment/markdown statements as appropriate. Non-working code or unnecessary code will be penalized. 

Submit your Jupyter file using the link below or provide a link to your Google Colab or Github file.


In [1]:
# import packages to use
import pandas as pd
import pycountry

In [2]:
# load dataframe
df = pd.read_excel("BookingDotCom_HotelReviews.xlsx", sheet_name="Data")

# sample the first 1000 rows of df
df = df[:1000]

# rename df columns to lower case
df.columns= df.columns.str.lower()

,hotel_name,hotel_address,review_count,non_review_scoring_count,average_hotel_score,review_date,reviewer_nationality,positive_comments,negative_comments,total_reviewer_reviews,reviewer_score,tags
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Russia,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Ireland,No real complaints the hotel was great great ...,No Negative,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double..."


In [3]:
'''
Create column with country name; we get the list of countries from the pycountry package
and use the hotel_address column to extract country name
'''

df['country'] = df["hotel_address"].apply(
    lambda address: ' '.join([c.name for c in pycountry.countries if c.name in address])
    )

# use the review_date column to extract the year and store in new column
df['year'] = pd.DatetimeIndex(df['review_date']).year

In [4]:
'''
In this step we deal with the tags column using the steps defined below:
    1. Define tags we are interested in
    2. Define a function to apply to tags column to remove tags we are not interested in by:
            - Converting the individual row values to list (from string) e.g. "[' Leisure trip ']" -> [' leisure trip ']
            - Strip the whitespaces from individual elements e.g. [' leisure trip ']-> ['leisure trip']
            - Drop tags we are not interested in
'''

# customer tags we are interested in
customer_tags = [
    'solo traveler',
    'group',
    'business trip',
    'leisure trip',
    'couple',
    'family with young children'
]

def clean_tag(x):

    # convert value from string to a list
    import ast
    
    myTags = ast.literal_eval(x.lower())

    # strip whitespaces from elements and drop those we are not interested in
    myTags = [customerTag.strip() for customerTag in myTags if customerTag.strip() in customer_tags]

    return myTags


# apply function to the tags column
df['tags'] = df['tags'].apply(lambda x: clean_tag(x))

In [5]:
df.head(2)

,hotel_name,hotel_address,review_count,non_review_scoring_count,average_hotel_score,review_date,reviewer_nationality,positive_comments,negative_comments,total_reviewer_reviews,reviewer_score,tags,country,year
0,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Russia,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,7,2.9,"[leisure trip, couple]",Netherlands,2017
1,Hotel Arena,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,1403,194,7.7,2017-08-03,Ireland,No real complaints the hotel was great great ...,No Negative,7,7.5,"[leisure trip, couple]",Netherlands,2017


In [6]:
'''
Function that to apply to df.tags column to extract new columns for customer categories
Lambda function will be used to apply this function to the column as in the steps below
'''
def split_tag(x:list, tagName:str) -> int:
    
    t = [1 if tagName in x else 0][0]
    
    return t

# dictionary of column names and customer tags (as contained in df.tags values)
tagDict = {
    'solo_traveler' : 'solo traveler',
    'group' : 'group',
    'business_trip' : 'business trip',
    'leisure_trip' : 'leisure trip',
    'couple' : 'couple',
    'family_with_young_children' : 'family with young children'
}

# applying function on tags column to get new separated columns
for key, value in tagDict.items():
    df[key] = df['tags'].apply(lambda x: split_tag(x, value))

In [7]:
df.columns

Index(['hotel_name', 'hotel_address', 'review_count',
       'non_review_scoring_count', 'average_hotel_score', 'review_date',
       'reviewer_nationality', 'positive_comments', 'negative_comments',
       'total_reviewer_reviews', 'reviewer_score', 'tags', 'country', 'year',
       'solo_traveler', 'group', 'business_trip', 'leisure_trip', 'couple',
       'family_with_young_children'],
      dtype='object')

In [8]:
# drop columns we do not need for now
colsToDrop = [
    'hotel_address', 
    'review_date', 
    'reviewer_nationality', 
    'tags'
    ]

df.drop(columns=colsToDrop, inplace=True)

In [10]:
df.head(2)

,hotel_name,review_count,non_review_scoring_count,average_hotel_score,positive_comments,negative_comments,total_reviewer_reviews,reviewer_score,country,year,solo_traveler,group,business_trip,leisure_trip,couple,family_with_young_children
0,Hotel Arena,1403,194,7.7,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...,7,2.9,Netherlands,2017,0,0,0,1,1,0
1,Hotel Arena,1403,194,7.7,No real complaints the hotel was great great ...,No Negative,7,7.5,Netherlands,2017,0,0,0,1,1,0
